# Update IAM Roles and Policies

In [1]:
import boto3
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

## Get SageMaker Execution Role Name

In [2]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

Role name: AmazonSageMaker-ExecutionRole-20240215T152311


In [3]:
setup_iam_roles_passed = False

# **Pre-Requisite:  SageMaker notebook instance ExecutionRole contains `AdministratorAccess` Policy.**
_Note:  The permissions used here are for demonstration purposes only.  Please follow least-privilege security principals appropriate for your environment._

In [4]:
admin = False

post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        setup_iam_roles_passed = True
        print("[OK] You are all set up to continue with this workshop!")
        break

if not admin:   
        print("*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************")

[OK] You are all set up to continue with this workshop!


# **If you see an ERROR message ^^ above ^^, please attach the AdministratorAccess Policy to the SageMaker notebook instance ExecutionRole.**
_Note:  The permissions used here are for demonstration purposes only.  Please follow least-privilege security principals appropriate for your environment._

# *Final Check*

In [6]:
# role = iam.get_role(RoleName=role_name)
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

required_policies = [
    "AdministratorAccess",
#     "SecretsManagerReadWrite",
#     "IAMFullAccess",
#     "AmazonS3FullAccess",
#     "AmazonAthenaFullAccess",
#     "ComprehendFullAccess",
#     "AmazonEC2ContainerRegistryFullAccess",
#     "AmazonRedshiftFullAccess",
#     "AWSStepFunctionsFullAccess",
#     "AmazonSageMakerFullAccess",
#     "AmazonKinesisFullAccess",
#     "AmazonKinesisFirehoseFullAccess",
#     "AmazonKinesisAnalyticsFullAccess",
]

admin = False

for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            break
    else:
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            pass

if not admin and len(required_policies) > 0:
    print("*************** [ERROR] RE-RUN THIS NOTEBOOK *****************")
    for required_policy in required_policies:
        print("Not Attached: {}".format(required_policy))
else:
    setup_iam_roles_passed = True
    print("[OK] You are all set up to continue with this workshop!")

[OK] You are all set up to continue with this workshop!


In [7]:
%store setup_iam_roles_passed

Stored 'setup_iam_roles_passed' (bool)


In [8]:
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True
setup_iam_roles_passed                -> True
setup_s3_bucket_passed                -> True


# Release Resources

In [9]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}